# 0. Design

### 0_1. 데이터 정의
- 입력 데이터는 1.csv, 2.csv, 3.csv, ... 와 같이 n.csv 형식의 파일
    - 입력 데이터: 악보 정보 (노트, 강도, 페달 등)
- 정답 데이터는 1_target.csv, 2_target.csv, 3_target.csv, ... 와 같이 n_target.csv 형식의 파일
    - 정답 데이터: 각 기준별 평가 점수
    
### 0_2. 데이터 전처리 방법
- input data
    - sec: 가만히 유지
    - msg_type: note_on, note_off 정보는 딥러닝 학습에 무의미하므로 삭제
    - channel: 삭제
    - note: MinMaxScaler (노트값은 데이터 분포가 정규 분포를 따르지 않을 가능성 ↑. 추가 실험 예정)
    - velocity: StandardScaler (velocity는 일정 정규 분포를 따를 가능성 ↑. 추가 실험 예정)
    - dynamic: One-Hot Encoding (각 강도 값을 고유한 벡터로 변환하여 모델이 더 쉽게 학습할 수 있도록 합니다.)
    - padal: StandardScaler
    - count: 삭제 (음악 정보와 관련성이 낮다고 판단)
    - main_vol: 삭제 (음악 정보와 관련성이 낮다고 판단)
    - depth: 삭제 (음악 정보와 관련성이 낮다고 판단)
    - pan: 삭제 (음악 정보와 관련성이 낮다고 판단)
    
### 0_3. 모델 선택 및 이유
- RNN & LSTM
    - 본 연구의 경우 입력 데이터의 행의 수가 연주된 곡의 길이에 따라 전부 다름(시간 의존성 내포)
    - 피아노 연주의 경우 이전의 연주가 다음의 연주와 이어지는 Sequence Data이므로 이를 처리하는데 적합한 RNN 모델을 선택
    - 또한, RNN 모델의 경우 긴 시퀀스 데이터의 경우 vanishing gradient 혹은 exploding gradient 문제가 발생하기 때문에 LSTM 모델을 선택
        - LSTM 모델은 셀 내부에 게이트 메커니즘을 사용하여 과거 정보를 유지하고 불필요한 정보를 잊도록 설계되어 이러한 문제를 해결

# 1. Import Libraries

In [19]:
from keras.models import Sequential
from keras.layers import SimpleRNN, LSTM, Dense, Embedding
from keras.preprocessing.sequence import pad_sequences

from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import random
import os

# 2. Data Load & Preprocessing

In [125]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import numpy as np
import pandas as pd

def load_data(filenames):
    input_data = []
    target_data = []

    scaler_note = MinMaxScaler()
    scaler_velocity = MinMaxScaler()

    for filename in filenames:
        df = pd.read_csv('./data/input/' + filename)

        # Drop unnecessary columns
        df.drop(['msg_type', 'channel', 'count', 'main_vol', 'depth', 'pan'], axis=1, inplace=True)

        # Convert 'note' and 'velocity' columns from string to list and handle empty strings
        df['note'] = df['note'].apply(lambda x: [int(i) for i in x.strip('[]').split(', ') if i != ''])
        df['velocity'] = df['velocity'].apply(lambda x: [int(i) for i in x.strip('[]').split(', ') if i != ''])

        # Replace empty lists with [0] for 'note' and 'velocity' columns
        df['note'] = df['note'].apply(lambda x: x if x else [0])
        df['velocity'] = df['velocity'].apply(lambda x: x if x else [0])

        # Normalize 'note' and 'velocity' columns
        df['note'] = df['note'].apply(lambda x: scaler_note.fit_transform(np.array(x).reshape(-1, 1)).flatten())
        df['velocity'] = df['velocity'].apply(lambda x: scaler_velocity.fit_transform(np.array(x).reshape(-1, 1)).flatten())

        # One-hot encode 'dynamic' column if it exists
        if 'dynamic' in df.columns:
            encoder = OneHotEncoder(sparse=False, categories='auto', drop=None, handle_unknown='ignore')
            encoded_dynamic = encoder.fit_transform(df[['dynamic']])
            # Convert one-hot encoding to NumPy array
            encoded_dynamic_array = np.array(encoded_dynamic)
            df.drop(['dynamic'], axis=1, inplace=True)
            df['dynamic'] = encoded_dynamic_array.tolist()

        # Convert dataframe to numpy array
        data = df.to_numpy()

        # Append data for each file
        input_data.append(data)

        # Load target data
        target_filename = filename.replace('.csv', '_target.csv')
        df_target = pd.read_csv('./data/target/' + target_filename)
        target_data.append(df_target.to_numpy())

    return input_data, target_data

filenames = ['1.csv', '2.csv', '3.csv'] # 나중에 반복문으로 자동화

input_data, target_data = load_data(filenames)

print('Sample from input data files:')
print(f'The number of input data: {len(input_data)}')
print(input_data[0][:10])
print(input_data[0].shape)
print(input_data[1].shape)
print(input_data[2].shape)

print()

print('Sample from target data files:')
print(f'The number of input data: {len(input_data)}')
print(target_data[0][0])
print(target_data[1][0])
print(target_data[2][0])
print(target_data[0].shape)
print(target_data[1].shape)
print(target_data[2].shape)

Sample from input data files:
The number of input data: 3
[[0.0 array([1., 0.]) array([0., 1.]) 0 0 list([1.0, 0.0, 0.0, 0.0, 0.0])]
 [0.1 array([0.]) array([0.]) 0 0 list([0.0, 0.0, 0.0, 0.0, 1.0])]
 [0.2 array([0.]) array([0.]) 0 0 list([0.0, 0.0, 0.0, 0.0, 1.0])]
 [0.3 array([0.]) array([0.]) 0 0 list([0.0, 0.0, 0.0, 0.0, 1.0])]
 [0.4 array([1.        , 0.        , 0.96296296, 0.25925926])
  array([0.        , 0.        , 0.80681818, 1.        ]) 0 0
  list([0.0, 0.0, 1.0, 0.0, 0.0])]
 [0.5 array([0.]) array([0.]) 0 0 list([0.0, 0.0, 0.0, 0.0, 1.0])]
 [0.6 array([0.]) array([0.]) 0 0 list([0.0, 0.0, 0.0, 0.0, 1.0])]
 [0.7 array([0.]) array([0.]) 0 0 list([0.0, 0.0, 0.0, 0.0, 1.0])]
 [0.8 array([0.]) array([0.]) 0 0 list([0.0, 0.0, 0.0, 0.0, 1.0])]
 [0.9 array([0.95, 0.  , 1.  , 0.25])
  array([0.        , 0.        , 0.87777778, 1.        ]) 0 0
  list([0.0, 0.0, 1.0, 0.0, 0.0])]]
(3594, 6)
(133, 6)
(133, 6)

Sample from target data files:
The number of input data: 3
[45.  64.  66. 

C:\Users\User\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [53]:
os.makedirs("./data/train/input", exist_ok=True)
os.makedirs("./data/train/target", exist_ok=True)
os.makedirs("./data/val/input", exist_ok=True)
os.makedirs("./data/val/target", exist_ok=True)
os.makedirs("./data/test/input", exist_ok=True)
os.makedirs("./data/test/target", exist_ok=True)

In [54]:
input_data_dir = "./data/input"
target_data_dir = "./data/target"
input_filenames = os.listdir(input_data_dir)

# target_filenames 생성
target_filenames = [filename.replace('.csv', '_target.csv') for filename in input_filenames]

print(input_filenames)
print(target_filenames)

train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

random.shuffle(input_filenames)

train_filenames = input_filenames[:int(len(input_filenames) * train_ratio)]
val_filenames = input_filenames[int(len(input_filenames) * train_ratio):int(len(input_filenames) * (train_ratio + val_ratio))]
test_filenames = input_filenames[int(len(input_filenames) * (train_ratio + val_ratio)):]

for filename in train_filenames:
    input_filepath = os.path.join(input_data_dir, filename)
    target_filepath = os.path.join(target_data_dir, target_filenames[input_filenames.index(filename)])
    if os.path.exists(target_filepath):  # 대응되는 타겟 파일이 존재하는 경우에만 이동
        os.rename(input_filepath, os.path.join("./data/train/input", filename))
        os.rename(target_filepath, os.path.join("./data/train/target", target_filenames[input_filenames.index(filename)]))
        print(f'{filename}이 train 폴더로 이동, target 파일 동일')

for filename in val_filenames:
    input_filepath = os.path.join(input_data_dir, filename)
    target_filepath = os.path.join(target_data_dir, target_filenames[input_filenames.index(filename)])
    if os.path.exists(target_filepath):  # 대응되는 타겟 파일이 존재하는 경우에만 이동
        os.rename(input_filepath, os.path.join("./data/val/input", filename))
        os.rename(target_filepath, os.path.join("./data/val/target", target_filenames[input_filenames.index(filename)]))
        print(f'{filename}이 val 폴더로 이동, target 파일 동일')

for filename in test_filenames:
    input_filepath = os.path.join(input_data_dir, filename)
    target_filepath = os.path.join(target_data_dir, target_filenames[input_filenames.index(filename)])
    if os.path.exists(target_filepath):  # 대응되는 타겟 파일이 존재하는 경우에만 이동
        os.rename(input_filepath, os.path.join("./data/test/input", filename))
        os.rename(target_filepath, os.path.join("./data/test/target", target_filenames[input_filenames.index(filename)]))
        print(f'{filename}이 test 폴더로 이동, target 파일 동일')

['1.csv', '2.csv', '3.csv']
['1_target.csv', '2_target.csv', '3_target.csv']
2.csv이 train 폴더로 이동, target 파일 동일
1.csv이 train 폴더로 이동, target 파일 동일
3.csv이 test 폴더로 이동, target 파일 동일


In [31]:
# Split data into training and test sets (80% train, 20% test)
filenames = ['1.csv', '2.csv', '3.csv']  # Replace with actual filenames

np.random.shuffle(filenames)

split_index = int(0.8 * len(filenames))

# Training filenames
train_filenames = filenames[:split_index]

# Test filenames
test_filenames = filenames[split_index:]

# Load training data
x_train, y_train = load_data(train_filenames)

# Load test data
x_test, y_test = load_data(test_filenames)

# 3. Implement Models

In [5]:
num_features = x_data.shape[1]
num_outputs = y_data.shape[1]

### 3_1. RNN

In [6]:
# 1. Create a RNN model
model = Sequential([
    Embedding(input_dim=num_features, output_dim=128),
    # Use the SimpleRNN layer for a basic RNN
    SimpleRNN(128, activation='relu', return_sequences=True),
    SimpleRNN(128, activation='relu'),
    Dense(num_outputs)
])

# 2. Summarization
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 128)         768       
                                                                 
 simple_rnn (SimpleRNN)      (None, None, 128)         32896     
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 128)               32896     
                                                                 
 dense (Dense)               (None, 6)                 774       
                                                                 
Total params: 67334 (263.02 KB)
Trainable params: 67334 (263.02 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [206]:
# 3. Train the model
model.compile(loss = 'mse', optimizer = 'adam', metrics = ['mae'])
model.fit(x_train, y_train, epochs=100)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

In [190]:
# 4. Evaluate the model
loss, mae = model.evaluate(x_test, y_test)
print('Loss:', loss)
print('MAE:', mae)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

### 3_2. LSTM

In [118]:
# 1. Create a LSTM model
model = Sequential([
    Embedding(input_dim=num_features, output_dim=128),
    LSTM(128, activation='relu', return_sequences=True),
    LSTM(128, activation='relu'),
    Dense(num_outputs)
])

# 2. Summarization
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_17 (Embedding)    (None, None, 128)         1536      
                                                                 
 lstm_2 (LSTM)               (None, None, 128)         131584    
                                                                 
 lstm_3 (LSTM)               (None, 128)               131584    
                                                                 
 dense_9 (Dense)             (None, 6)                 774       
                                                                 
Total params: 265478 (1.01 MB)
Trainable params: 265478 (1.01 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [119]:
# 3. Train the model
model.compile(loss = 'mse', optimizer = 'adam', metrics = ['mae'])
model.fit(x_train, y_train, epochs=100)

NameError: name 'x_train' is not defined

In [120]:
# 4. Evaluate the model
loss, mae = model.evaluate(x_test, y_test)
print('Loss:', loss)
print('MAE:', mae)

NameError: name 'x_test' is not defined